In [35]:
!tree -C ../../src/tutorial/grpc/geodatas

../../src/tutorial/grpc/geodatas
├── __init__.py
├── common
│   ├── __init__.py
│   └── base.py
├── models
│   ├── __init__.py
│   ├── ogrgeojson.py
│   └── thing.py
├── proto
│   ├── __init__.py
│   ├── search.proto
│   ├── search_pb2.py
│   └── search_pb2_grpc.py
├── rpc
│   ├── __init__.py
│   └── search_servicer.py
├── search_client.py
├── search_server.py
└── tools
    ├── __init__.py
    ├── geometry.py
    ├── rpc_stub.py
    └── service_discovery.py

5 directories, 18 files


# Search Client

In [ ]:
../../src/tutorial/grpc/geodatas
├── __init__.py
├── common
│   ├── __init__.py
│   └── base.py
├── models
│   ├── __init__.py
│   ├── ogrgeojson.py
│   └── thing.py
├── proto
│   ├── __init__.py
│   ├── search.proto
│   ├── search_pb2.py      ((*))
│   └── search_pb2_grpc.py ((*))
├── rpc
│   ├── __init__.py
│   └── search_servicer.py (*)
├── search_client.py (*)
├── search_server.py
└── tools
    ├── __init__.py
    ├── geometry.py
    ├── rpc_stub.py (*)
    └── service_discovery.py (*)


- (*)   Files focus
- ((*)) Generated files

## Application

In [33]:
# %load ../../src/tutorial/grpc/geodatas/search_client.py
import argparse
import logging

from tutorial.grpc.geodatas.proto import search_pb2_grpc, search_pb2
from tutorial.grpc.geodatas.tools.rpc_stub import get_search_rpc_stub_with_consul

logger = logging.getLogger(__name__)


def process_request_position(search_service_stub: search_pb2_grpc.SearchStub, lat: float, lng: float):
    [...]
    

def process(args):
    [...]


def main():
    args = parse_arguments()
    process(args)

ModuleNotFoundError: No module named 'tutorial.grpc.geodatas.tools.rpc_stub'

### Processing - Consul (as services registry)

In [ ]:
# %load ../../src/tutorial/grpc/geodatas/search_client.py
from tutorial.grpc.geodatas.proto import search_pb2_grpc, search_pb2
from tutorial.grpc.geodatas.tools.rpc_stub import get_search_rpc_stub_with_consul

logger = logging.getLogger(__name__)


def process_request_position(search_service_stub: search_pb2_grpc.SearchStub, lat: float, lng: float):
    [...]
    

def process(args):
    search_service_stub = get_search_rpc_stub_with_consul(args.consul_resolver_port,
                                                          args.consul_resolver_nameservers,
                                                          args.consul_resolver_nameservice)

    if args.monitor:
        process_monitor(search_service_stub)

    if args.request_position_latlng:
        process_request_position(search_service_stub, *args.request_position_latlng)

#### Tools - Consul + gRPC stub

In [ ]:
# %load ../../src/tutorial/grpc/geodatas/tools/rpc_stub.py
from grpc import insecure_channel
from tutorial.grpc.geodatas.proto import search_pb2_grpc
from tutorial.grpc.geodatas.tools.service_discovery import find_service_with_consul


def get_search_rpc_stub(ip: str, port: int) -> search_pb2_grpc.SearchStub:
    logger.info(f"creating grpc client based on consul data: ip={ip} port={port}")
    channel = grpc.insecure_channel(f'{ip}:{port}')
    stub = search_pb2_grpc.SearchStub(channel)
    return stub


def get_search_rpc_stub_with_consul(consul_resolver_port: int,
                                    consul_resolver_nameservers: List[str],
                                    consul_resolver_nameservice: str) -> search_pb2_grpc.SearchStub:
    search_service_ip, search_service_port = find_service_with_consul(consul_resolver_port,
                                                                      consul_resolver_nameservers,
                                                                      consul_resolver_nameservice)

    return get_search_rpc_stub(search_service_ip, search_service_port)

### Processing - Consul (as service registry)

In [ ]:
# %load ../../src/tutorial/grpc/geodatas/search_client.py
from tutorial.grpc.geodatas.proto import search_pb2_grpc, search_pb2
from tutorial.grpc.geodatas.tools.rpc_stub import get_search_rpc_stub_with_consul

logger = logging.getLogger(__name__)


def process_request_position(search_service_stub: search_pb2_grpc.SearchStub, lat: float, lng: float):
    [...]
    

def process(args):
    search_service_stub = get_search_rpc_stub_with_consul(args.consul_resolver_port,
                                                          args.consul_resolver_nameservers,
                                                          args.consul_resolver_nameservice)

    if args.monitor:
        process_monitor(search_service_stub)

    if args.request_position_latlng:
        process_request_position(search_service_stub, *args.request_position_latlng)

### Processing - Request position

In [ ]:
# %load ../../src/tutorial/grpc/geodatas/search_client.py
from tutorial.grpc.geodatas.proto import search_pb2_grpc, search_pb2
from tutorial.grpc.geodatas.tools.rpc_stub import get_search_rpc_stub_with_consul

logger = logging.getLogger(__name__)


def process_request_position(search_service_stub: search_pb2_grpc.SearchStub, lat: float, lng: float):
    req = search_pb2.SearchRequest(query="search_client", lat=lat, lng=lng, result_per_page=10)
    logger.debug("sending request: {}".format(req))
    resp = search_service_stub.search(req)
    logger.debug("received response: {}".format(resp if resp else 'None'))
    

def process(args):
    [...]

In [ ]:
# %load ../../src/tutorial/grpc/geodatas/proto/search_pb2_grpc.py
class SearchStub(object):
    def __init__(self, channel):
        self.monitor = [...]
        self.search = channel.unary_unary(
            '/Search/search',
            request_serializer=tutorial_dot_grpc_dot_geodatas_dot_proto_dot_search__pb2.SearchRequest.SerializeToString,
            response_deserializer=tutorial_dot_grpc_dot_geodatas_dot_proto_dot_search__pb2.SearchResponses.FromString,
            )

## Execution logs of Search Client

In [ ]:
$ python search_client.py --request_position_latlng 4.850000 45.750000 --m
2019-02-24 10:06:35,599 - __main__ - INFO - creating grpc client based on consul data: ip=127.0.0.1 port=54813
2019-02-24 10:06:35,612 - __main__ - DEBUG - monitor response: n_things: 4

2019-02-24 10:06:35,612 - __main__ - DEBUG - sending request: query: "search_client"
result_per_page: 10
lat: 4.849999904632568
lng: 45.75

2019-02-24 10:06:35,630 - __main__ - DEBUG - received response: responses {
  response: "69123 - Lyon - fr:Lyon"
}